# Tally slurm total GPU hours for an account

This assumes that you have run

```bash
sacct -A pli --format=JobID,Account,AllocTRES%30,Elapsed  --allusers --json > sacct_pli.json    
```

on the server, printing out all jobs for the `pli` account.

In [27]:
import json
from pathlib import Path
from datetime import timedelta

In [8]:
json_data = Path("..", 'data',  'sacct_pli.json')

assert json_data.exists()

In [10]:
data = json.loads(json_data.read_text())

In [35]:
def get_n_gpus(job_data: dict) -> int:
    n_gpus = 0
    for allocation in job_data["tres"]["allocated"]:
        if allocation["type"] == "gres" and allocation["name"] == "gpu":
            n_gpus += int(allocation["count"])
    return n_gpus

def get_total_run_time(job_data: dict) -> timedelta:
    seconds = job_data["time"]["elapsed"]
    return timedelta(seconds=seconds)

In [33]:
total_gpu_time = timedelta(seconds=0)
for job in data["jobs"]:
    total_gpu_time += get_n_gpus(job) * get_total_run_time(job)

In [34]:
total_gpu_time

datetime.timedelta(days=39, seconds=78792)